# Lecture 3

Let's try to do something simple in python and introduce some particle physics basics. In relativistic mechanics, the Energy and Momentum of particles are different in every frame, but obey $m^2=E^2-\vec{p}^2$ or $E^2=\vec{p}^2$ for massless particles, where we set the speed of light $c=1$. It is therefore convenient to express the Energy and Momentum of a particle as a 4-vector, for example in Euclidean coordiates: $p= (E,p_{x},p_{y},p_{z}) = (E,\vec{p})$.

Energy and momentum are concerved with a particle decays into two other particles, for example a $Z$ boson to two electrons, $Z\rightarrow e^+ e^-$, or a Higgs Boson to two photons, $H\rightarrow \gamma\gamma$. In 4-vectors we can express conservations, for example in the Higgs decay, as $p_H = p_{\gamma1}+p_{\gamma2}$. In a two body decay, it's easy to fully solve for the momenta daughter particles in the rest frame of the parent:

$$
m_H = 1.25 GeV\\
p_H = (m_{H},0,0,0)
$$

Momentum conservation tells us that $\vec{p_{H}} = 0 = \vec{p_{\gamma1}} + \vec{p_{\gamma2}} \Rightarrow \vec{p_{\gamma1}} = - \vec{p_{\gamma2}} = p_\gamma$, i.e. the daughters travel in opposite directions. The 4-vector of the photons are

$$
E_H = m_H = E_{\gamma1}+E_{\gamma2} = |\vec{p_{\gamma1}}| + |\vec{p_{\gamma2}}|=2|p_\gamma|\\
\Rightarrow p_{\gamma1}= (m_H/2, \vec{p_{\gamma}})\\
\Rightarrow p_{\gamma2}= (m_H/2, -\vec{p_{\gamma}})
$$

If we select that direction to be aligned with one of our axes, then we can write:
$$
p_{\gamma1}= (m_H/2, 0,0, m_H/2)\\
p_{\gamma2}= (m_H/2, 0,0, -m_H/2).
$$

We can compute these 4-vectors in the case that the parent particle is not at rest by relavistic boosting. 

We will begin by representing 4-vectors as python lists. For example the first photon in the rest frame can be written as:

In [1]:
m_H= 125.
p_g1= [m_H/2,0,0,m_H/2]
print p_g1

[62.5, 0, 0, 62.5]


To get the second photon, lets write a function that negates 4-vectors:

In [2]:
def neg_4v(p):
    return [p[0], -p[1],  -p[2] , -p[3]]

p_g2=neg_4v(p_g1)
print p_g2

[62.5, 0, 0, -62.5]


Other useful functions:

In [3]:
def add_v4(p1,p2):
    return [p1[0]+p2[0], p1[1]+p2[1],  p1[2]+p2[2] , p1[3]+p2[3]]

def sub_v4(p1,p2):
    return add_v4(p1,neg_4v(p2))

print "Sum:", add_v4(p_g1,p_g2)
print "Difference:", sub_v4(p_g1,p_g2)

Sum: [125.0, 0, 0, 0.0]
Difference: [125.0, 0, 0, 125.0]


In [4]:
import math

def dot_v4(p1,p2):
    return math.sqrt(sum([p1[0]*p2[0], -p1[1]*p2[1],  -p1[2]*p2[2] , -p1[3]*p2[3]]))

def mass_v4(p):
    return dot_v4(p,p)
    
print "Dot:", dot_v4(p_g1,p_g2)
print "Gamma Mass:", mass_v4(p_g1)
print "Higgs Mass:", mass_v4(add_v4(p_g1,p_g2))

Dot: 88.3883476483
Gamma Mass: 0.0
Higgs Mass: 125.0


There are lots of ways to write the same thing, for example:

In [5]:
def add_v4_1(p1,p2):
    out=list()
    for i in range(4):
        out.append(p1[i]+p2[i])
    return out

def add_v4_2(p1,p2):
    return map(lambda x: x[0]+x[1],zip(p1,p2))

def add_v4_3(p1,p2):
    return [sum(x) for x in zip(p1,p2)]

print "zip:", zip(p_g1,p_g2)

print "Sum:", add_v4_3(p_g1,p_g2)

zip: [(62.5, 62.5), (0, 0), (0, 0), (62.5, -62.5)]
Sum: [125.0, 0, 0, 0.0]


In [43]:
def boost_matrix(beta_in):
    Lambda= [[0,0,0,0],
             [0,0,0,0],
             [0,0,0,0],
             [0,0,0,0]]
    
    beta=[0]+beta_in

    beta2=sum(x**2 for x in beta)
    gamma=1./math.sqrt(1.-beta2)
    
    for i in range(4):
        for j in range(4):
            if j==0:
                Lambda[i][0]=-gamma*beta[i]
            elif i==0:
                Lambda[0][j]=-gamma*beta[j]
            else:
                Lambda[i][j]= (gamma-1)*beta[i]*beta[j]/beta2 + float(i==j)

    Lambda[0][0]=gamma

    return Lambda
                
def boost(p,beta):
    Lambda=boost_matrix(beta)
    out=4*[0.]
    for j in range(4):
        out[j]=sum(map(lambda x: x[0]*x[1],zip(p,Lambda[j])))
    return out

def decay(p):
    m=mass_v4(p)
    p1=[m/2.,0.,0.,m/2.]
    p2=[m/2.,0.,0.,-m/2.]
    # We should now rotate by 2 arbitrary angles...
    beta=[p[1]/p[0],p[2]/p[0],p[3]/p[0]]
    
    p1b=boost(p1,beta)
    p2b=boost(p2,beta)

    return p1b,p2b


In [44]:
# Start with a Higgs at rest
p_H=[m_H,0.,0.,0.]

# Now boost it (along y for example)
p_Hb=boost(p_H,[0.,.5,0.])
print "Boosted Higgs:", p_Hb
print "Mass of Boosted Higgs:", mass_v4(p_Hb)

# Decay the boosted Higgs
p1,p2=decay(p_Hb)

# Make sure the decay products add back to the Higgs
print "Higgs from daughters:", add_v4(p1,p2)
print "Higgs mass from daughters:", mass_v4(add_v4(p1,p2))


Boosted Higgs: [144.33756729740645, 0.0, -72.16878364870323, 0.0]
Mass of Boosted Higgs: 125.0
Higgs from daughters: [144.33756729740648, 0.0, 72.16878364870324, 0.0]
Higgs mass from daughters: 125.0


Lets write a function that gives us the 4-vectors of 2 daughter particles given a parent particle 4 vector.

## Object Oriented Programming

Lets write a 4-vector class to do the same thing:

In [45]:
class four_vector(object):
    def __init__(self, p=None):
        if p:
            self.v=p
        else:
            self.v=[0.,0.,0.,0.]

    def setval(self,l):
        self.v=l
            
    def __add__(self,other):
        return four_vector([sum(x) for x in zip(self.v,other)])
    
    def neg(self,p):
        return four_vector([p[0], -p[1],  -p[2] , -p[3]])

    def __sub__(self,other):
        return self.__add__(self.v,self.neg(other))
  
    def __mul__(self,other):
        return math.sqrt(sum([self.v[0]*other[0], 
                              -self.v[1]*other[1],
                              -self.v[2]*other[2],
                              -self.v[3]*other[3]]))

    def boost(self,beta):
        Lambda=boost_matrix(beta)
        out=4*[0.]
        for j in range(4):
            out[j]=sum(map(lambda x: x[0]*x[1],zip(self.v,Lambda[j])))
        return four_vector(out)

    def mass(self):
        return self.__mul__(self.v)

    def __getitem__(self,i):
        return self.v[i]

    
    def __str__(self):
        return "({0}, {1}, {2}, {3})".format(self.v[0],self.v[1],self.v[2],self.v[3])


        

In [46]:
def decay(p):
    m=p.mass()
    p1=four_vector([m/2.,0.,0.,m/2.])
    p2=four_vector([m/2.,0.,0.,-m/2.])
    # We should now rotate by 2 arbitrary angles...
    beta=[p[1]/p[0],p[2]/p[0],p[3]/p[0]]
    
    p1b=p1.boost(beta)
    p2b=p2.boost(beta)

    return p1b,p2b



In [47]:
# Start with a Higgs at rest
p_H=four_vector([m_H,0.,0.,0.])
print "Initial Higgs:", p_H

# Now boost it (along y for example)
p_Hb=p_H.boost([0.,.5,0.])
print "Boosted Higgs:", p_Hb
print "Mass of Boosted Higgs:", p_Hb.mass()

# Decay the boosted Higgs
p1,p2=decay(p_Hb)

# Make sure the decay products add back to the Higgs
print "Higgs from daughters:", p1+p2
print "Higgs mass from daughters:", (p1+p2).mass()

Initial Higgs: (125.0, 0.0, 0.0, 0.0)
Boosted Higgs: (144.337567297, 0.0, -72.1687836487, 0.0)
Mass of Boosted Higgs: 125.0
Higgs from daughters: (144.337567297, 0.0, 72.1687836487, 0.0)
Higgs mass from daughters: 125.0


In [48]:
class composite(four_vector):
    def __init__(self,daughters):
        super(composite, self).__init__()
        self.daughters=daughters

        tmp=four_vector()
        for d in self.daughters:
            tmp=tmp+d
          
        self.setval(tmp.v)




In [49]:
H_reco=composite([p1,p2])
print "Composite Higgs:", H_reco
print "Mass:", H_reco.mass()

Composite Higgs: (144.337567297, 0.0, 72.1687836487, 0.0)
Mass: 125.0


# Dictionaries

In [54]:
Events=[]

for i in range(1,11):
    p_H=four_vector([m_H,0.,0.,0.])
    my_boost= float(i)/11.
    p_Hb=p_H.boost([0.,my_boost,0.])
    p1,p2=decay(p_Hb)
    
    Event= {"Higgs":composite([p1,p2]),
           "Boost":my_boost}
    
    Events.append(Event)
    
# Make sure the decay products add back to the Higgs
for i,Event in enumerate(Events):
    print "Event:",i
    print "Higgs 4-vector:",Event["Higgs"]
    print "Boost:",Event["Boost"]



Event: 0
Higgs 4-vector: (125.519752762, 0.0, 11.4108866147, 0.0)
Boost: 0.0909090909091
Event: 1
Higgs 4-vector: (127.118794968, 0.0, 23.1125081761, 0.0)
Boost: 0.181818181818
Event: 2
Higgs 4-vector: (129.925287597, 0.0, 35.4341693446, 0.0)
Boost: 0.272727272727
Event: 3
Higgs 4-vector: (134.18626003, 0.0, 48.7950036474, 0.0)
Boost: 0.363636363636
Event: 4
Higgs 4-vector: (140.335349847, 0.0, 63.788795385, 0.0)
Boost: 0.454545454545
Event: 5
Higgs 4-vector: (149.13968975, 0.0, 81.348921682, 0.0)
Boost: 0.545454545455
Event: 6
Higgs 4-vector: (162.045304022, 0.0, 103.119738923, 0.0)
Boost: 0.636363636364
Event: 7
Higgs 4-vector: (182.123199096, 0.0, 132.453235707, 0.0)
Boost: 0.727272727273
Event: 8
Higgs 4-vector: (217.406589137, 0.0, 177.878118384, 0.0)
Boost: 0.818181818182
Event: 9
Higgs 4-vector: (300.049599074, 0.0, 272.772362795, 0.0)
Boost: 0.909090909091
